In [ ]:
from extract.scraper import Scraper
from extract.common import config


In [ ]:
scraper = Scraper()
scraper.scrape('cumplo')

In [ ]:
import pymongo
from bson.json_util import dumps

host = config()['mongodb']['host']
user = config()['mongodb']['user']
password = config()['mongodb']['password']

uri = "mongodb+srv://{}:{}@{}/test?retryWrites=true".format(user,password,host)
client = pymongo.MongoClient(uri)
db = client['investment']
project = db['project']

In [ ]:
project.count()